In [1]:
import numpy as np
import xarray as xr
import pandas as pd

hadisst = xr.open_dataset('~/Science/lgm_foram_census/tidy/HadISST_PI.nc').sst
tierneysst = xr.open_dataset('~/Science/lgm_foram_census/tidy/Tierney2020_DA_ocn_regrid.nc').SSTLGM

## the hadisst data has a strange bug that cannot be directly slice (frozen dimension)
## here is a workaround
formatted_hadisst = xr.DataArray(
    data=hadisst.values,
    dims=('lat', 'lon'),
    coords={'lat': hadisst.latitude.values, 'lon': hadisst.longitude.values},
)

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# OBSERVATION DATA
forcens = pd.read_csv("~/Science/lgm_foram_census/tidy/forcens_sp_r_wsst.csv")
lgm = pd.read_csv("~/Science/lgm_foram_census/tidy/lgm_sp_r_wsst.csv")
margo = lgm[lgm["Data_Source"]=='margo']

topt = pd.read_csv("../data/obs_sp_Topt.csv")
species_list = topt.species

In [3]:
def hab_lat(df, species):
    ## select latitudes, longitudes and species columns
    df_sub = df[['Latitude','Longitude',species]]

    ## filter out the species with 0 abundance
    df_sub = df_sub[df_sub[species] > 0]
    
    ## get the habitat lattiude
    sp_hab_lat =  df_sub.Latitude
    sp_hab_north = sp_hab_lat[sp_hab_lat > 0]
    sp_hab_south = sp_hab_lat[sp_hab_lat < 0]
    
    ## get the habitat lattiude min and max in North and South hemispehre
    sp_hab_north_min = sp_hab_north.min()
    sp_hab_north_max = sp_hab_north.max()
    sp_hab_south_min = sp_hab_south.min()
    sp_hab_south_max = sp_hab_south.max()    
    
    return sp_hab_north_min, sp_hab_north_max, sp_hab_south_min, sp_hab_south_max

def hab_sstdiff(species):
    ## get the habitat lattiude range and use this to get the habitat sst

    ## get the PI sst
    hab_north_min, hab_north_max, hab_south_min, hab_south_max = hab_lat(forcens, species)
    pi_hab_north_sst = np.nanmean(formatted_hadisst.sel(lat=slice(hab_north_max, hab_north_min)))
    pi_hab_south_sst = np.nanmean(formatted_hadisst.sel(lat=slice(hab_south_max, hab_south_min)))
    pi_hab_sst = (pi_hab_south_sst+ pi_hab_north_sst)/2

    ## get the LGM sst
    hab_north_min, hab_north_max, hab_south_min, hab_south_max = hab_lat(margo, species)
    ## the order in the slice function seems to be reversed in the LGM
    lgm_hab_north_sst = tierneysst.sel(lat=slice(hab_north_min,hab_north_max)).mean().values
    lgm_hab_south_sst = tierneysst.sel(lat=slice(hab_south_min,hab_south_max)).mean().values
    lgm_hab_sst = (lgm_hab_south_sst+ lgm_hab_north_sst)/2

    diff = pi_hab_sst - lgm_hab_sst

    return diff

In [4]:
## use the func to get the delta temp
diff = []
for sp in species_list:
    diff.append(hab_sstdiff(sp))

In [5]:
# merge back to the topt df
topt["delta_habtemp"] = diff

topt = topt[["species","delta_habtemp"]]
topt.to_csv("../data/obs_sp_hab.csv", index=False)